## Spotify part - getting the related artists

In [6]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import pandas as pd

In [7]:
# Establish a session and instantiate it
client_credentials_manager = SpotifyClientCredentials(client_id='ccae2e7a3a0f48088d24563289eb71d0',
                                                      client_secret='0204fbff5f2c438a816b7c01ede9d9ce')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [8]:
artist_name = 'Jorge Drexler'
related_artists = []
result = sp.search(q='artist:' + artist_name, type='artist')
try:
    name = result['artists']['items'][0]['name']
    uri = result['artists']['items'][0]['uri']

    related = sp.artist_related_artists(uri)
    print('Related artists for', name)
    for artist in related['artists']:
        related_artists.append(artist['name'])
    print(related_artists)
except:
    print("Error!")

Related artists for Jorge Drexler
['Kevin Johansen', 'Pedro Guerra', 'Fito Paez', 'Luis Alberto Spinetta', 'Pedro Aznar', 'Ismael Serrano', 'Martín Buscaglia', 'Ana Prada', 'Mercedes Sosa', 'Silvio Rodríguez', 'Joaquín Sabina', 'Fabiana Cantilo', 'Sui Generis', 'Luis Eduardo Aute', 'Charly García', 'Lisandro Aristimuño', 'Pescado Rabioso', 'Serú Girán', 'Andrés Calamaro', 'Onda Vaga']


## Genius part - getting the lyrics for each

In [10]:
import requests
import json
import requests
from bs4 import BeautifulSoup
import re

CLIENT_ACCESS_TOKEN = "lTv0ZCyw1mkXQ7_9bZ9GmEdc9vPh2_VQzhFCpERSOYiFA1fXjEVrI8BY-D9_k6E2"
BASE_URI = "https://api.genius.com"

In [12]:
#from: https://gist.github.com/imdkm/a60247b59ff1881fa4bb8846a9b44c96
# send request and get response in json format.
def _get(path, params=None, headers=None):

    # generate request URL
    requrl = '/'.join([BASE_URL, path])
    token = "Bearer {}".format(CLIENT_ACCESS_TOKEN)
    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}

    response = requests.get(url=requrl, params=params, headers=headers)
    response.raise_for_status()

    return response.json()


for artist_name in related_artists:
    print("searching " + artist_name + "'s artist id. \n")

    # find artist id from given data.
    find_id = _get("search", {'q': artist_name})
    for hit in find_id["response"]["hits"]:
       if hit["result"]["primary_artist"]["name"] == artist_name:
           artist_id = hit["result"]["primary_artist"]["id"]
           break

    print("-> " + artist_name + "'s id is " + str(artist_id) + "\n")

searching Kevin Johansen's artist id. 



NameError: name 'BASE_URL' is not defined